## **Lab 02: Simple Linear Regression를 Tensorflow로 구현하기**

### **Build hypothesis and cost**

![test](./img/lb2-1.png)

**머신러닝 학습은 cost가 최소화가 되는 W(가중치)와 b(편향)을 찾는것이다.**

In [2]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()

In [3]:
tf.__version__

'1.14.0'

In [15]:
# cost
# cost = tf.reduce_mean(tf.square(hypothesis - y_data))
# tf.reduce_mean(): rank(차원)가 줄어들면서 mean을 구한다.
# v = [1.,2.,3.,4.]
# tf.reduce_mean(v) # 2.5
# tf.square()
# tf.square(3) # 9

### **경사 하강법(Gradient Descent)소개**

**W 파라미터의 개수가 적다면 고차원 방정식으로 비용 함수가 최소가 되는 W 변숫값을 도출할 수 있겠지만, W 파라미터가 많으면 고차원 방정식을 동원하더라도 해결하기가 어렵다.**

**경사 하강법은 이러한 고차원 방정식에 대한 문제를 해결해 주면서 비용 함수 RSS를 최소화하는 방법을 직관적으로 제공하는 뛰어난 방식이다.**

**사실 경사 하강법은 '데이터를 기반으로 알고리즘이 스스로 학습한다'는 머신러닝의 개념을 가능하게 만들어준 핵심 기법의 하나이다.**

**경사 하강법의 사전적 의미인 '점진적인 하강'이라는 뜻에서도 알 수 있듯이, '점진적으로' 반복적인 계산을 통해 W 파라미터 값을 업데이트하면서 오류 값이 최소가 되는 W 파라미터를 구하는 방식이다.**

> **어떻게 보면 무식해 보이는 방법이지만 W 파라미터의 개수에 따라 매우 복잡해지는 고차원 방정식을 푸는 것보다 훨씬 더 직관적이고 빠르게 비용 함수가 최소가 되는 W파라미터 값을 구할 수 있다.**

**경사 하강법은 반복적으로 비용 함수의 반환 값, 즉 예측값과 실제 값의 차이가 작아지는 방향성을 가지고 W 파라미터를 지속해서 보정해 나간다.**

> **최초 오류 값이 100이었다면 두 번째 오류 값은 100보다 작은 90, 세 번째는 80과 같은 방식으로 지속해서 오류를 감소시키는 방향으로 W 값을 계속 업데이트해 나간다.**

> **그리고 오류 값이 더 이상 작아지지 않으면 그 오류 값을 최소 비용으로 판단하고 그때의 W값을 최적 파라미터로 반환한다.**

**경사 하강법의 핵심은 "어떻게 하면 오류가 작아지는 방향으로 W 값을 보정할 수 있을까?"이다.**

> **예를 들어 비용 함수가 포물선 형태의 2차 함수라면 경사 하강법은 최초 W 에서 부터 미분을 적용한 뒤 이 미분 값이 계속 감소하는 방향으로 순차적으로 W를 업데이트한다.**

> **더 이상 미분된 1차 함수의 기울기가 감소하지 않는 지점을 비용 함수가 최소인 지점으로 간주하고 그때의 W를 반환한다.**

![test](./img/경사하강법.png)

![test](./img/경사하강법1.png)

> **step1: w1를 임의의 값으로 설정(bias 생략)하고 첫 비용함수의 값을 계산한다.**

> **step2: w1 을 위 그림과 같은 식으로 업데이트한 후 다시 비용 함수의 값을 계산한다.**

> **step3: 비용 함수의 값이 감소했으면 다시 step 2를 반복한다. 더 이상 비용 함수의 값이 감소하지 않으면 그때의 w1를 구하고 반복을 중지한다.**

In [20]:
# Data set
x_data = [1,2,3,4,5]
y_data = [1,2,3,4,5]

# init weight, bias
W = tf.Variable(2.9)
b = tf.Variable(0.5)

# H(x) = Wx + b
hypothesis = W*x_data + b

# Gradient descent
# learning_rate initialize
learning_rate = 0.01

# Gradient descent를 총 100번을 반복하기 위한 코드
for i in range(100): 
    # Gradient descent(W,b가 한번 업데이트를 위한 코드)
    # with구문 안에 있는 변수들의 정보를 tape에 저장하게 된다.
    with tf.GradientTape() as tape:
        hypothesis = W * x_data + b
        cost = tf.reduce_mean(tf.square(hypothesis - y_data))

    # cost값에 대한 개별 파라미터(w,b)의 미분 값을 w_grad, b_grad에 반환
    W_grad, b_grad = tape.gradient(cost, [W,b])

    # Weight update
    # W(업데이트된 Weight) -= learning_rate * W(현재의 W의 기울기)
    W.assign_sub(learning_rate * W_grad)
    b.assign_sub(learning_rate * b_grad)
    
    # 10의 배수가 될때 마다 중간 값들을 확인
    if i % 10 == 0:
        print('{:5}|{:10.4f}|{:10.4f}|{:10.6f}'.format(i,W.numpy(),b.numpy(),cost))

# Predict
print(W * 5 + b)
print(W * 2.5 + b)

    0|    2.4520|    0.3760| 45.660004
   10|    1.1036|    0.0034|  0.206336
   20|    1.0128|   -0.0209|  0.001026
   30|    1.0065|   -0.0218|  0.000093
   40|    1.0059|   -0.0212|  0.000083
   50|    1.0057|   -0.0205|  0.000077
   60|    1.0055|   -0.0198|  0.000072
   70|    1.0053|   -0.0192|  0.000067
   80|    1.0051|   -0.0185|  0.000063
   90|    1.0050|   -0.0179|  0.000059
tf.Tensor(5.0066934, shape=(), dtype=float32)
tf.Tensor(2.4946523, shape=(), dtype=float32)


![test](./img/lb2-2.png)

**초기값 W = 2.9, b = 0.5에서 반복수를 1로 하여 한번 학습된 회귀직선을 보면 기존 데이터에 제대로 학습되지 않은 결과를 볼 수 있고, 경사하강법을 통해 반복수를 90번으로 했을 때 오른쪽 그림처럼 기존 데이터를 잘 학습하여 최적의 회귀직선이 구해짐을 알 수 있다.**